<a href="https://colab.research.google.com/github/s2t2/tweet-analysis-2021/blob/main/notebooks/Exporting_Disinfo_User_Timeline_Tweets_to_CSV_20240325.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Exporting data to drive...

Earlier we only exported the "original" tweets (excluding RTs, mentions, and replies). Now let's also export the remainder. There are 11M tweets total. If we store in Parquert it might be ok.

Using "High RAM" runtime on Colab.

### BigQuery Service

In [2]:
from google.colab import auth

auth.authenticate_user()

In [3]:
#from google.auth import default
#creds, _ = default()
#print(creds)

In [4]:
from google.cloud import bigquery
from pandas import DataFrame, read_gbq

#PROJECT_ID = "tweet-collector-py"
PROJECT_ID = "tweet-research-shared"

class BigQueryService():
    def __init__(self, project_id=PROJECT_ID):
        self.project_id = project_id
        self.client = bigquery.Client(project=self.project_id)

    def execute_query(self, sql, verbose=True):
        if verbose == True:
            print(sql)
        job = self.client.query(sql)
        return job.result()

    #def query_to_df(self, sql, verbose=True):
    #    """high-level wrapper to return a DataFrame"""
    #    results = self.execute_query(sql, verbose=verbose)
    #    return DataFrame([dict(row) for row in results])

    def query_to_df(self, sql, verbose=True):
        """high-level wrapper to return a DataFrame"""
        if verbose == True:
            print(sql)
        # https://pandas.pydata.org/docs/reference/api/pandas.read_gbq.html#pandas-read-gbq
        return read_gbq(sql, project_id=self.project_id, progress_bar_type="tqdm_notebook")


In [5]:
bq = BigQueryService()
print(bq)

In [ ]:
print("DATASETS:")
datasets = list(bq.client.list_datasets())

In [7]:
for ds in datasets:
    #print("...", ds.project, ds.dataset_id)
    if "user" not in ds.dataset_id:
        print("...", ds.reference)

... tweet-research-shared.analysis_2021
... tweet-research-shared.disinfo_2021
... tweet-research-shared.election_2020
... tweet-research-shared.election_2020_transition_2021_combined
... tweet-research-shared.f1_racing_2023
... tweet-research-shared.impeachment_2020
... tweet-research-shared.impeachment_2021
... tweet-research-shared.transition_2021
... tweet-research-shared.truth_2023


# Usage

Fetch table of "original" timeline tweets (excludes rt, reply, quotes):

In [8]:
print("------------")
print("QUERY:")
sql = """
    SELECT user_id, status_id, status_text, geo, created_at, lookup_at
        ,truncated
        ,is_quote
        ,reply_status_id , reply_user_id, is_reply
        ,retweeted_status_id, retweeted_user_id, retweeted_user_screen_name ,is_rt
    FROM `tweet-research-shared.disinfo_2021.q_user_timeline_tweets`
"""

df = bq.query_to_df(sql, verbose=True)
df

------------
QUERY:

    SELECT user_id, status_id, status_text, geo, created_at, lookup_at
        ,truncated
        ,is_quote
        ,reply_status_id , reply_user_id, is_reply
        ,retweeted_status_id, retweeted_user_id, retweeted_user_screen_name ,is_rt
    FROM `tweet-research-shared.disinfo_2021.q_user_timeline_tweets`



Downloading:   0%|          |

,user_id,status_id,status_text,geo,created_at,lookup_at,truncated,is_quote,reply_status_id,reply_user_id,is_reply,retweeted_status_id,retweeted_user_id,retweeted_user_screen_name,is_rt
0,2733971311,1369380656766197771,"RT @TAFEeducation: Be straight with us, Minist...",None,2021-03-09 20:12:45+00:00,2021-03-28 07:47:04+00:00,False,True,<NA>,<NA>,False,1369225839246602241,2336306731,TAFEeducation,True
1,2734066113,1325520341553336321,RT @TomFitton: Tom Fitton: @RealDonaldTrump wo...,None,2020-11-08 19:27:31+00:00,2021-02-13 16:23:42+00:00,False,False,<NA>,<NA>,False,1325485291352813568,18266688,TomFitton,True
2,2734066113,1325939383971753987,RT @MattRinaldiTX: This guy is on @JoeBiden’s ...,None,2020-11-09 23:12:38+00:00,2021-02-13 16:23:42+00:00,False,False,<NA>,<NA>,False,1325864840846598147,354041953,MattRinaldiTX,True
3,2734066113,1335319784821563396,RT @atensnut: Will Ruby Freeman be allowed any...,None,2020-12-05 20:27:00+00:00,2021-02-13 16:23:39+00:00,False,False,<NA>,<NA>,False,1335316050406232064,22677397,atensnut,True
4,2734066113,1351972312275443713,RT @K1erry: WaPo Finally Admits Trump DIDN'T I...,None,2021-01-20 19:18:12+00:00,2021-02-13 16:23:36+00:00,False,False,<NA>,<NA>,False,1351934625992630272,418867782,K1erry,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12009810,809801490705612800,1355149896014127112,RT @DonaldJTrumpJr: Any Republican in Washingt...,None,2021-01-29 13:44:47+00:00,2021-04-12 13:20:49+00:00,False,False,<NA>,<NA>,False,1354886480674619394,39344374,DonaldJTrumpJr,True
12009811,811250837465006081,1368930001198911488,RT @DonaldJTrumpJr: Imagine how many more ther...,None,2021-03-08 14:22:00+00:00,2021-05-06 02:08:24+00:00,False,True,<NA>,<NA>,False,1368713033661485057,39344374,DonaldJTrumpJr,True
12009812,1067365741,1360288352927445002,RT @DonaldJTrumpJr: Trump impeachment trial --...,None,2021-02-12 18:03:10+00:00,2021-02-14 03:23:05+00:00,False,False,<NA>,<NA>,False,1360287173560438786,39344374,DonaldJTrumpJr,True
12009813,1069437140184981504,1364303479720747011,RT @DonaldJTrumpJr: You knew it was coming!!! ...,None,2021-02-23 19:57:51+00:00,2021-04-03 22:44:47+00:00,False,False,<NA>,<NA>,False,1364197165141291011,39344374,DonaldJTrumpJr,True


In [9]:
df.columns.tolist()

['user_id',
 'status_id',
 'status_text',
 'geo',
 'created_at',
 'lookup_at',
 'truncated',
 'is_quote',
 'reply_status_id',
 'reply_user_id',
 'is_reply',
 'retweeted_status_id',
 'retweeted_user_id',
 'retweeted_user_screen_name',
 'is_rt']

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12009815 entries, 0 to 12009814
Data columns (total 15 columns):
 #   Column                      Dtype              
---  ------                      -----              
 0   user_id                     Int64              
 1   status_id                   Int64              
 2   status_text                 object             
 3   geo                         object             
 4   created_at                  datetime64[ns, UTC]
 5   lookup_at                   datetime64[ns, UTC]
 6   truncated                   boolean            
 7   is_quote                    boolean            
 8   reply_status_id             Int64              
 9   reply_user_id               Int64              
 10  is_reply                    boolean            
 11  retweeted_status_id         Int64              
 12  retweeted_user_id           Int64              
 13  retweeted_user_screen_name  object             
 14  is_rt                       bool

In [11]:
df.dtypes

user_id                                     Int64
status_id                                   Int64
status_text                                object
geo                                        object
created_at                    datetime64[ns, UTC]
lookup_at                     datetime64[ns, UTC]
truncated                                 boolean
is_quote                                  boolean
reply_status_id                             Int64
reply_user_id                               Int64
is_reply                                  boolean
retweeted_status_id                         Int64
retweeted_user_id                           Int64
retweeted_user_screen_name                 object
is_rt                                     boolean
dtype: object

In [12]:
df.iloc[0]["status_text"]

'RT @TAFEeducation: Be straight with us, Minister @GeoffLeeMP. Will you or won’t you sell Bega TAFE? Seems you close a local TAFE college ev…'

In [13]:
users_pivot = df.groupby("user_id")["status_id"].count().sort_values(ascending=False)
users_pivot

user_id
1226463888             6309
759855850496389120     6022
1211080316070195201    5760
4372515753             5002
1196634077526417414    4784
                       ... 
2423655841               25
1168940599346683905      21
835537291405832192       19
713829760959381504        6
1115685588944875520       2
Name: status_id, Length: 4718, dtype: int64

In [14]:
# drop duplicates:

print(len(df))
df.drop_duplicates(subset=["status_id"], keep="last", inplace=True)
print(len(df))

12009815
11982573


### Exporting to Drive

In [15]:
import os
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [16]:
# you might need to create a google drive SHORTCUT that has this same path
# ... or update the path to use your own google drive organization
DIRPATH = '/content/drive/MyDrive/Research/DS Research Shared 2024'
os.path.isdir(DIRPATH)

True

In [17]:
DATA_DIRPATH = os.path.join(DIRPATH, "projects", "Disinformation 2021 Embeddings", "data")
os.path.isdir(DATA_DIRPATH)

True

In [18]:
#csv_filepath = os.path.join(DATA_DIRPATH, "q_user_timeline_tweets.csv.gz")
#df.to_csv(csv_filepath, index=False, compression="gzip")

In [19]:
pq_filepath = os.path.join(DATA_DIRPATH, "q_user_timeline_tweets.parquet.gz")

df.to_parquet(pq_filepath, index=False, compression="gzip")